In [1]:
%install-location $cwd/swift-install
%install '.package(path: "../../..")' Epochs

Installing packages:
	.package(path: "../../..")
		Epochs
With SwiftPM flags: []
Working in: /tmp/tmp3o5h9y5d/swift-install
/home/sgugger/swift/usr/bin/swift-build: /home/sgugger/anaconda3/lib/libcurl.so.4: no version information available (required by /home/sgugger/swift/usr/lib/swift/linux/libFoundationNetworking.so)
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
import TensorFlow
import Epochs

In [3]:
// Base use
// Some raw items (for instance filenames)
let rawItems = 0..<512
// A heavy-compute function lazily mapped on it (for instance, opening the images)
let dataSet = rawItems.lazy.map { _ in Tensor<Float>(randomNormal: [224, 224, 3]) }
// A `Batches` defined on this:
let batches = Batches(of: 64, from: dataSet, \.collated)
// Iteration over it:
for batch in batches {
    print(batch.shape)
}

[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]


In [4]:
// Enabling shuffle
let batches = Batches(of: 64, from: dataSet.shuffled(), \.collated)
// This should absolutely not be done this way because it traverses the collection:
print(type(of: batches))

Batches<Array<Tensor<Float>>, Tensor<Float>>


In [5]:
// We need to actually go back to raw collection:
let dataSet = rawItems.shuffled().lazy.map { _ in Tensor<Float>(randomNormal: [224, 224, 3]) }
let batches = Batches(of: 64, from: dataSet, \.collated)
print(type(of: batches))

Batches<LazyMapSequence<Array<Int>, Tensor<Float>>, Tensor<Float>>


In [5]:
// Use with padding
// Let's create an array of things of various lengths (for instance texts)
var dataSet: [Tensor<Int32>] = []
for _ in 0..<512 {
    dataSet.append(Tensor<Int32>(
        randomUniform: [Int.random(in: 1...200)], 
        lowerBound: Tensor<Int32>(0), 
        upperBound: Tensor<Int32>(100)
    ))
}

// We need to pad those tensors to make them all the same length.
// We could do this in one lazy transform applied beforehand and pad everything
// to the same length, but it's not memory-efficient: some batches might need less
// padding. So we need to add the padding after having selected the samples we
// are trying to batch.
let batches = Batches(of: 64, from: dataSet) { $0.collatedAndTailPadded(with: 0) }
for b in batches {
    print(b.shape)
}

[64, 199]
[64, 198]
[64, 200]
[64, 196]
[64, 196]
[64, 196]
[64, 197]
[64, 198]


In [6]:
// Use with a sampler
// In our previous example, another way to be memory efficient is to batch
// samples of roughly the same lengths.
let sortedDataset = dataSet.sorted { $0.shape[0] > $1.shape[0] }

let batches = Batches(of: 64, from: sortedDataset) { $0.collatedAndTailPadded(with: 0) }
for b in batches {
    print(b.shape)
}

[64, 200]
[64, 179]
[64, 153]
[64, 128]
[64, 102]
[64, 81]
[64, 54]
[64, 29]


In [7]:
// When using a `batchSize` we get a bit of shuffle:
let sortedDataset = dataSet.shuffled().sortedInBatches(of: 256) { $0.shape[0] > $1.shape[0] }

let batches = Batches(of: 64, from: sortedDataset) { $0.collatedAndTailPadded(with: 0) }
for b in batches {
    print(b.shape)
}

[64, 199]
[64, 150]
[64, 99]
[64, 56]
[64, 200]
[64, 154]
[64, 102]
[64, 51]


In [8]:
struct LanguageModelDataset<Texts: RandomAccessCollection> where Texts.Element == [Int] {
    /// The underlying collection of texts
    public var texts: Texts
    /// The length of the samples returned when indexing
    private let sequenceLength: Int
    // The texts all concatenated together
    private var stream: [Int]
    
    init(texts: Texts, sequenceLength: Int) {
        self.texts = texts
        self.sequenceLength = sequenceLength
        stream = texts.reduce([], +)
    }
}

In [9]:
extension LanguageModelDataset: RandomAccessCollection {
    public typealias Index = Int
    public typealias Element = Tensor<Int32>
    
    public var startIndex: Int { return 0 }
    public var endIndex: Int { return stream.count / sequenceLength }
    public func index(after i: Int) -> Int { i+1 }
    
    public subscript(index: Int) -> Tensor<Int32> {
        get { 
            let i = index * sequenceLength
            return Tensor<Int32>(stream[i..<i+sequenceLength].map { Int32($0)} )
        }
    }
}

In [10]:
//Let's create such a DataSet
let numbers: [[Int]] = [[1,2,3,4,5], [6,7,8], [9,10,11,12,13,14,15], [16,17,18]]
var dataset = LanguageModelDataset(texts: numbers, sequenceLength: 3)

In [11]:
//Now let's look at what it gives us:
let batches = Batches(of: 3, from: dataset, \.collated)
for b in batches {
    print(b)
}

[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]]
[[10, 11, 12],
 [13, 14, 15],
 [16, 17, 18]]


In [12]:
var dataset = LanguageModelDataset(texts: numbers.shuffled(), sequenceLength: 3)
let batches = Batches(of: 3, from: dataset, \.collated)
for b in batches {
    print(b)
}

[[ 9, 10, 11],
 [12, 13, 14],
 [15,  1,  2]]
[[ 3,  4,  5],
 [16, 17, 18],
 [ 6,  7,  8]]


## `BatchesGenerator`

In [13]:
// Base use
// Some raw items (for instance filenames)
let rawItems = 0..<512
// A heavy-compute function lazily mapped on it (for instance, opening the images)
let trainingSet = Samples(on: Array(0..<512)) {  $0.lazy.map { _ in Tensor<Float>(randomNormal: [224, 224, 3]) } }
let validationSet = Samples(on: Array(0..<256)) {  $0.lazy.map { _ in Tensor<Float>(randomNormal: [224, 224, 3]) } }

In [14]:
// A `Batches` defined on this:
let batchesGenerator = BatchesGenerator(of: 64, from: trainingSet, and: validationSet, \.collated)

In [15]:
let (trainingBatches, validationBatches) = batchesGenerator.nextEpoch()
for b in trainingBatches { print(b.shape) }
for b in validationBatches { print(b.shape) }

[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
